# Reasearch Track II Assignment 2

---

**Author's Name: Omotoye Shamsudeen Adekoya** <br>
**Student ID: 5066348**

---

## NoteBook Description

This Notebook has two functions; it serves as a user interface for the control of non-holonomic robot in a 3D environment and it also show a graphical feedback of the robot state and behaviour. 

In [22]:
import ipywidgets as widgets
from ipywidgets import Button, Layout, HBox, VBox
from ipywidgets import Button, GridBox, Layout, ButtonStyle
%matplotlib widget


## Robot Control

In [23]:
import matplotlib.pyplot as plt
import rospy
import tf
from nav_msgs.msg import Odometry
import numpy as np
from matplotlib.animation import FuncAnimation
from tf.transformations import euler_from_quaternion
from math import *

class Visualiser:
    def __init__(self):
        self.fig, self.ax = plt.subplots()
        self.ln, = plt.plot([], [], 'ro')
        self.x_data, self.y_data = [] , []

    def plot_init(self):
        self.ax.set_xlim(-5, 5)
        self.ax.set_ylim(-5, 5)
        return self.ln
    def odom_callback(self, msg):
        self.y_data.append(msg.pose.pose.position.y)
        self.x_data.append(msg.pose.pose.position.x)

    def update_plot(self, frame):
        self.ln.set_data(self.x_data, self.y_data)
        return self.ln
    

In [24]:
import rospy
import time
from rt2_assignment2.srv import Command, RandomPosition
from geometry_msgs.msg import Twist
from nav_msgs.msg import Odometry
#from rt2_assignment2.msg import TargetPose 


from all_buttons import *

from ipywidgets import Button, Layout, HBox, VBox
plt.style.use('ggplot')

def start_simulation(a):
    ui_client("start")
    start_button.button_style = 'success'
    stop_button.button_style = ''
    
def stop_simulation(b):
    ui_client("stop")
    start_button.button_style = ''
    stop_button.button_style = 'danger'  
    
def move_forward(a):
    if (rospy.has_param('/linear_velocity')):
        lv_multiplier = rospy.get_param('/linear_velocity')
    else:
        lv_multiplier = 1
    vel = Twist()
    vel.linear.x = lv_multiplier*0.5
    pub.publish(vel)
    up_button.button_style = 'info'
    down_button.button_style = ''
    left_button.button_style = ''
    right_button.button_style = ''
    stop.button_style = ''
    
    
def move_backward(a):
    if (rospy.has_param('/linear_velocity')):
        lv_multiplier = rospy.get_param('/linear_velocity')
    else:
        lv_multiplier = 1
    vel = Twist()
    vel.linear.x = lv_multiplier*(-0.5)
    pub.publish(vel)
    up_button.button_style = ''
    down_button.button_style = 'info'
    left_button.button_style = ''
    right_button.button_style = ''
    stop.button_style = ''
  

def turn_left(a):
    if (rospy.has_param('/angular_velocity')):
        av_multiplier = rospy.get_param('/angular_velocity')
    else:
        av_multiplier = 1
    vel = Twist()
    vel.angular.z = av_multiplier*(-0.5)
    pub.publish(vel)
    up_button.button_style = ''
    down_button.button_style = ''
    left_button.button_style = 'info'
    right_button.button_style = ''
    stop.button_style = ''
   

def turn_right(a):
    if (rospy.has_param('/angular_velocity')):
        av_multiplier = rospy.get_param('/angular_velocity')
    else:
        av_multiplier = 1
    vel = Twist()
    vel.angular.z = av_multiplier*(0.5)
    pub.publish(vel)
    up_button.button_style = ''
    down_button.button_style = ''
    left_button.button_style = ''
    right_button.button_style = 'info'
    stop.button_style = ''

    
def stop_movement(a):
    vel = Twist()
    vel.linear.x = 0.0
    vel.angular.z = 0.0
    pub.publish(vel)
    up_button.button_style = ''
    down_button.button_style = ''
    left_button.button_style = ''
    right_button.button_style = ''
    stop.button_style = 'danger'
    
    
def on_value_change(change):
    x = change['new']
    rospy.set_param('/linear_velocity', x)
    
    
def on_valueang_change(change):
    z = change['new']
    rospy.set_param('/angular_velocity', z)
    
    
def get_analysis_data():
    if (rospy.has_param('/reached_target')):
        reached_target = rospy.get_param('/reached_target')
    else: 
        reached_target = 0
    if (rospy.has_param('/canceled_target')):
        canceled_target = rospy.get_param('/canceled_target')
    else:
        canceled_target = 0
    if (rospy.has_param('/target_time')):
        time_to_target = rospy.get_param('/target_time')
    else:
        time_to_target = []
    return reached_target, canceled_target, time_to_target


def button_event():
    start_button.on_click(start_simulation)
    stop_button.on_click(stop_simulation)
    up_button.on_click(move_forward)
    down_button.on_click(move_backward)
    left_button.on_click(turn_left)
    right_button.on_click(turn_right)
    stop.on_click(stop_movement)
    l_velSlider.observe(on_value_change, names='value')
    a_velSlider.observe(on_valueang_change, names='value')
    
def main():
    rospy.init_node('user_interface')
    button_event()

if __name__ == '__main__':
    rospy.wait_for_service('/user_interface')
    try:
        ui_client = rospy.ServiceProxy('/user_interface', Command)
    except rospy.ServiceException as e:
        print(f'Service call failed: {e}')
    main()
    vis = Visualiser()
    pub = rospy.Publisher('/cmd_vel', Twist, queue_size=10)
    sub = rospy.Subscriber('/odom', Odometry, vis.odom_callback)
    ani = FuncAnimation(vis.fig, vis.update_plot, init_func=vis.plot_init)
    plt.show(block=True)
    
VBox([l_velSlider, a_velSlider])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [25]:
HBox([start_button, stop_button])

In [26]:
display(control_pad)

GridBox(children=(Button(icon='arrow-up', layout=Layout(grid_area='up', height='50px', width='auto'), style=Bu…

In [38]:
reached_target, canceled_target, time_to_target = get_analysis_data()

plt.figure(dpi=100)
plt.hist(time_to_target,bins=50, stacked=True)
plt.figure(dpi=80, figsize=(9,2.2))
plt.barh(('Reached\nTarget', 'Canceled\nTarget'),(reached_target, canceled_target))
plt.show()

/tmp/ipykernel_1431/3287019885.py:3: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.figure(dpi=100)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/tmp/ipykernel_1431/3287019885.py:5: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.figure(dpi=80, figsize=(9,2.2))


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …